# 1. Introduction - How IKEA Retail Standardizes Docker Images for Efficient Machine Learning Model Deployment: A Practical Demonstration

---

This Jupyter notebook provides a comprehensive walkthrough of the technical processes discussed in the blog entry titled **"How IKEA Retail Standardizes Docker Images for Efficient Machine Learning Model Deployment."** You can find the original blog post [here](https://www.docker.com/blog/how-ikea-retail-standardizes-docker-images-for-efficient-machine-learning-model-deployment/).

Throughout this notebook, we will explore the following steps in detail:

1. **Data Ingestion**: How we source our data.
2. **Data Preparation**: Getting our data ready for modeling.
3. **Model Training**: Building our machine learning model.
4. **Model Evaluation**: Assessing the performance of our model.
5. **Model Preparation Before Deploying**: Ensuring the model is ready for real-world applications.
6. **DockerFile Creation**: Crafting the environment for our model.
7. **Model Deployment**: Getting our model live and operational.

Each section is elaborated with in-depth explanations and code demonstrations. For any queries or clarifications, please reach out to [fernandodorado.rueda@ingka.com](mailto:fernandodorado.rueda@ingka.com).

Let's dive in!


# 1. Import Dependencies

In this section, we will import all necessary libraries and modules that will be used throughout this notebook. Here's a brief overview of each:

- **os**: The built-in Python `os` module allows us to interact with the operating system, enabling tasks like reading or writing to files, creating directories, etc.

- **pickle**: This module is essential for serializing (saving) and deserializing (loading) Python objects, which is crucial for saving and loading trained machine learning models.

- **numpy**: Short for 'Numerical Python', `numpy` is fundamental for numerical operations in Python. It provides support for arrays (including multidimensional arrays), as well as a collection of mathematical functions to operate on arrays.

- **pandas**: A powerful library for data analysis and manipulation. It offers data structures for efficiently storing large datasets and tools for reshaping, aggregating, and filtering data.

- **sklearn (scikit-learn)**: One of the most popular machine learning libraries in Python. We'll be using several functionalities from `sklearn`:
  - **datasets**: For accessing sample datasets.
  - **train_test_split**: To split our data into training and test sets.
  - **LogisticRegression**: A machine learning algorithm for classification tasks.
  - **Perceptron**: Another algorithm for binary classification.

Let's proceed to import these dependencies.

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron

# 2. Data Loading and Pre-processing

In this section, we'll accomplish the following:

1. **Load the Dataset**: We'll be using the breast cancer dataset provided by scikit-learn. To align with our ethos of transparency and consumer privacy and to facilitate your engagement with this approach, a public dataset was employed.

2. **Data Splitting**: To evaluate the performance of our machine learning models, we need to split our data into a training set and a testing set.

3. **Export Test Data (Optional)**: Optionally, we can save our testing features (`X_test`) and labels (`y_test`) as separate CSV files for potential external evaluations or other purposes.

4. **Combine Features and Targets**: We will then combine our test features and labels into a single DataFrame for ease of access and visualization.

5. **Save Combined Test Data**: Finally, we'll save the combined test data to a CSV file. This can be especially handy for sharing or using the test set in different environments without having to repeat the pre-processing steps.

In [2]:
# Load the breast cancer dataset
X, y = datasets.load_breast_cancer(return_X_y=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0)

# Save X_test and y_test as separate CSV files (Optional)
# np.savetxt("X_test.csv", X_test, delimiter=",", header=",".join([f"feature_{i}" for i in range(X_test.shape[1])]), comments="")
# np.savetxt("y_test.csv", y_test, delimiter=",", header="target", comments="")

# Combine X_test and y_test into a single DataFrame
X_test_df = pd.DataFrame(X_test, columns=[f"feature_{i}" for i in range(X_test.shape[1])])
y_test_df = pd.DataFrame(y_test, columns=["target"])

df_test = pd.concat([X_test_df, y_test_df], axis=1)

# Save the combined DataFrame as a CSV file
df_test.to_csv("data/testing_data.csv", index=False)

# 3. Training Machine Learning Models

In this section, we'll focus on the following steps:

1. **Setting Up the Model Storage Path**: We define a path where our trained models will be stored. This makes it easier to retrieve, share, or deploy them later.

2. **Preparing the Storage Directory**: If the model storage directory doesn't already exist, we'll create it to ensure our models have a designated place to be saved.

3. **Model Configurations**: We will define configurations for two different classifiers: 
    - `Logistic Regression`: A popular algorithm for binary and multiclass classification tasks.
    - `Perceptron`: A binary classification algorithm inspired by neural networks.

4. **Training Process**: For each classifier, we will:
    - Train it using the training data.
    - Save the trained model to the specified file path using the `pickle` module.
    - Print a confirmation message indicating the model's storage location.

In [3]:
# Define the path to store models
model_path = "models/"

# Create the folder if it doesn't exist
if not os.path.exists(model_path):
    os.makedirs(model_path)

# Define a list of classifier parameters
parameters = [
    {"clf": LogisticRegression(solver="liblinear", multi_class="ovr"), "name": f"{model_path}/binary-lr.joblib"},
    {"clf": Perceptron(eta0=0.1, random_state=0), "name": f"{model_path}/binary-percept.joblib"},
]

# Iterate through each parameter configuration
for param in parameters:
    clf = param["clf"]  # Retrieve the classifier from the parameter dictionary
    clf.fit(X_train, y_train)  # Fit the classifier on the training data
    
    # Save the trained model to a file using pickle
    model_filename = f"{param['name']}"
    with open(model_filename, 'wb') as model_file:
        pickle.dump(clf, model_file)
    
    print(f"Model saved in {model_filename}")

Model saved in models//binary-lr.joblib
Model saved in models//binary-percept.joblib


# 4. Evaluating Machine Learning Models

After training our models, it's essential to evaluate their performance on unseen data to gauge their effectiveness and reliability. We do this using a variety of metrics, depending on the type of problem and the specific requirements of the project.

For this demonstration, we'll employ the following metrics:

1. **Accuracy**: The ratio of correctly predicted instances to the total number of instances. It gives a general overview of the model's performance.
   
2. **Precision**: The number of true positives divided by the sum of true positives and false positives. It's a measure of a model's relevance.
   
3. **Recall (Sensitivity)**: The number of true positives divided by the sum of true positives and false negatives. It measures the model's capability to identify all relevant instances.
   
4. **F1-score**: The harmonic mean of precision and recall. It's a balance between precision and recall and is particularly useful when class distributions are imbalanced.

In this section, we'll:
- Load a previously trained Logistic Regression model.
- Make predictions on the testing data using this model.
- Evaluate the model's performance using the aforementioned metrics

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Example of predictions
model_path = 'models/binary-lr.joblib'
with open(model_path, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Make predictions using the loaded model
predictions = loaded_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

# Calculate precision
precision = precision_score(y_test, predictions)

# Calculate recall
recall = recall_score(y_test, predictions)

# Calculate F1-score
f1 = f1_score(y_test, predictions)

print(f"Model loaded from {model_path}")
print(f"Accuracy on the testing dataset: {accuracy:.2f}")
print(f"Precision on the testing dataset: {precision:.2f}")
print(f"Recall on the testing dataset: {recall:.2f}")
print(f"F1-score on the testing dataset: {f1:.2f}")


Model loaded from models/binary-lr.joblib
Accuracy on the testing dataset: 0.93
Precision on the testing dataset: 0.95
Recall on the testing dataset: 0.93
F1-score on the testing dataset: 0.94


# 5. Model Deployment

The process of deploying machine learning models encompasses various intricate steps, from pre-processing the data and training the model to fine-tuning it for optimal performance and then integrating it into the final application. One of the critical junctures in this voyage is sculpting the Model Class, where Docker is key in these steps.


## 5.1. Creation Model Class

The Model Class stands as an architectural cornerstone when deploying machine learning models. Specifically designed for integration with deployment platforms such as Docker, this class upholds the high standards established by tools like Seldon.

### Loading and Predicting
* load(): A function that is tasked with importing the pretrained model from a designated path. This function is imperative for initializing the prediction process.

* predict(X, features_names=None, meta=None): Upon successful loading of the model, this function becomes operational, predicting outcomes based on the input features provided. The function can also accommodate optional parameters such as feature names and other metadata.

### Feedback Handling
* send_feedback(features, feature_names, reward, truth, routing=""): This function holds significant importance as it enables the model to recalibrate based on real-world feedback. By juxtaposing its predictions against the ground truth, the model's metrics receive real-time updates, facilitating iterative refinement of its performance.


### Metrics Calculation
* calculate_metrics(): This function is dedicated to evaluating the model's efficacy. Utilizing metrics such as accuracy, precision, recall, and F1-score, it provides a comprehensive overview of the model's operational efficiency.

* Score class: Nested within the DockerModel, this auxiliary class diligently monitors the essential metrics for binary classification – true positives, false positives, true negatives, and false negatives.

### Monitoring Assistance
* metrics(): This function is a linchpin for the continuous monitoring of the model. Crafted to be compliant with Prometheus metrics, it ensures real-time scrutiny and potential adjustments to the model's performance.

* tags(): Specifically crafted to extract prediction-associated metadata, this function is an asset for monitoring. Its output is paramount for debugging operations, offering insights into the nature of requests being processed by the model.

The DockerModel class epitomizes the seamless transition of machine learning models from developmental environments to production-grade deployments. Its architecture is pivotal in assuring deployment standardization, thereby infusing efficiency and consistency into the process.

With the foundation laid in the form of the Model Class, the subsequent phase involves the encapsulation of this model within a Docker image. This transition is characterized by a harmonious blend of technological prowess and strategic implementation, aiming to redefine ML deployment paradigms.

In [5]:
%%writefile DockerModel.py

import joblib
import logging

class Score:
    """
    Class to hold metrics for binary classification, including true positives (TP), false positives (FP),
    true negatives (TN), and false negatives (FN).
    """
    def __init__(self, TP=0, FP=0, TN=0, FN=0):
        self.TP = TP  # True Positives
        self.FP = FP  # False Positives
        self.TN = TN  # True Negatives
        self.FN = FN  # False Negatives


class DockerModel:
    """
    Class for loading and predicting using a pre-trained model, handling feedback to update metrics,
    and providing those metrics.
    """
    result = {}  # Dictionary to store input data

    def __init__(self, model_name="models/binary-lr.joblib"):
        """
        Initialize DockerModel with metrics and model name.
        :param model_name: Path to the pre-trained model.
        """
        self.scores = Score(0, 0, 0, 0)
        self.loaded = False
        self.model_name = model_name

    def load(self):
        """
        Load the model from the provided path.
        """
        self.model = joblib.load(self.model_name)
        logging.info(f"Model {self.model_name} Loaded")

    def predict(self, X, features_names=None, meta=None):
        """
        Predict the target using the loaded model.
        :param X: Features for prediction.
        :param features_names: Names of the features, optional.
        :param meta: Additional metadata, optional.
        :return: Predicted target values.
        """
        self.result['shape_input_data'] = str(X.shape)
        logging.info(f"Received request: {X}")
        if not self.loaded:
            self.load()
            self.loaded = True
        predictions = self.model.predict(X)
        return predictions

    def send_feedback(self, features, feature_names, reward, truth, routing=""):
        """
        Provide feedback on predictions and update the metrics.
        :param features: Features used for prediction.
        :param feature_names: Names of the features.
        :param reward: Reward signal, not used in this context.
        :param truth: Ground truth target values.
        :param routing: Routing information, optional.
        :return: Empty list as return value is not used.
        """
        predicted = self.predict(features)
        print(f"Predicted: {predicted[0]}, Truth: {truth[0]}")
        if int(truth[0]) == 1:
            if int(predicted[0]) == int(truth[0]):
                self.scores.TP += 1
            else:
                self.scores.FN += 1
        else:
            if int(predicted[0]) == int(truth[0]):
                self.scores.TN += 1
            else:
                self.scores.FP += 1
        return []  # Ignore return statement as its not used

    def calculate_metrics(self):
        """
        Calculate the accuracy, precision, recall, and F1-score.
        :return: accuracy, precision, recall, f1_score
        """
        total_samples = self.scores.TP + self.scores.TN + self.scores.FP + self.scores.FN

        # Check if there are any samples to avoid division by zero
        if total_samples == 0:
            logging.warning("No samples available to calculate metrics.")
            return 0, 0, 0, 0  # Return zeros for all metrics if no samples

        accuracy = (self.scores.TP + self.scores.TN) / total_samples

        # Check if there are any positive predictions to calculate precision
        positive_predictions = self.scores.TP + self.scores.FP
        precision = self.scores.TP / positive_predictions if positive_predictions != 0 else 0

        # Check if there are any actual positives to calculate recall
        actual_positives = self.scores.TP + self.scores.FN
        recall = self.scores.TP / actual_positives if actual_positives != 0 else 0

        # Check if precision and recall are non-zero to calculate F1-score
        if precision + recall == 0:
            f1_score = 0
        else:
            f1_score = 2 * (precision * recall) / (precision + recall)

        # Return the calculated metrics
        return accuracy, precision, recall, f1_score


    def metrics(self):
        """
        Generate metrics for monitoring.
        :return: List of dictionaries containing accuracy, precision, recall, and f1_score.
        """
        accuracy, precision, recall, f1_score = self.calculate_metrics()
        return [
            {"type": "GAUGE", "key": "accuracy", "value": accuracy},
            {"type": "GAUGE", "key": "precision", "value": precision},
            {"type": "GAUGE", "key": "recall", "value": recall},
            {"type": "GAUGE", "key": "f1_score", "value": f1_score},
        ]
        
    def tags(self):
        """
        Retrieve input data used for predictions.
        :return: Dictionary containing input data.
        """
        return self.result


Overwriting DockerModel.py


## 5.2 Local Testing: A Crucial Pre-Docker Step

Before diving into the creation of a Docker image, it is of paramount importance to undertake local testing of the model. This step serves as a precursor, ensuring the model's readiness for the subsequent phase.

### The Significance of Local Testing
Local testing is not just a procedural measure; it embodies the essence of quality assurance. It plays a pivotal role in:

* Error Identification: It assists in pinpointing issues at an early stage, thus circumventing potential challenges that could surface during the deployment phase.

* Performance Verification: Utilizing the code snippet provided below, it ascertains that the model is producing predictions in the anticipated format.

In [6]:
# Test Model Locally
from DockerModel import DockerModel

demoModel = DockerModel()

In [7]:
demoModel.predict(X_test)

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,

In [9]:
demoModel.send_feedback(X_test, truth = y_test, feature_names = None, reward = None)

Predicted: 0, Truth: 0


[]

## 6. Encapsulating the model into a Docker image

In the context of IKEA Retail MLOps, a model is not merely an assemblage of code. It represents a nuanced assembly encompassing code, dependencies, and ML artifacts, all encapsulated within a Docker image that's versioned and registered. Such a composition underscores the meticulous attention paid to the architecture of the physical infrastructure.


What Role Does Docker Play in MLOps?
* Docker emerges as a linchpin in MLOps, fostering a uniform environment that simplifies the transition from development to deployment:
* Streamlined Deployment: Docker containers encapsulate the entire prerequisites an ML model necessitates for execution, thereby facilitating the deployment journey.
* Enhanced Collaboration: Docker enables data scientists and engineers to maintain consistency in models and their dependencies throughout diverse development phases.
* Model Reproducibility: A consistent environment courtesy of Docker accentuates the reproducibility of models—a paramount consideration in machine learning.
* Orchestration Tool Integration: Docker's compatibility with orchestration platforms, such as Kubernetes, empowers automated deployment, scalability, and efficient containerized application management.

Docker and containerization transcend mere technological instruments; they act as catalysts propelling innovation and efficiency within MLOps. Championing consistency, scalability, and agility, Docker ushers in an era of refined ML deployment processes. For developers, data scientists, and IT professionals alike, a grasp over Docker is indispensable to navigate the intricate terrain of contemporary data-driven applications.

## 6.1. Crafting Dockerfile
Crafting a Dockerfile can be equated to drafting a building's architectural blueprint. It enumerates instructions to formulate a Docker image, ensuring the application's seamless execution within a pristine, isolated milieu. Such a delineation guarantees that the entirety of the model—including its code, dependencies, and unique ML artifacts—syncs harmoniously with the grand narrative of IKEA Retail's MLOps paradigm.

For our demonstration, a Dockerfile has been sculpted with the explicit intent to encapsulate both the model's code and its allied artifacts. This judicious architecture ensures a seamless transition from development to deployment.

### 6.2. Create requirements.txt file required for the Model Deployment

In [10]:
%%writefile requirements.txt

pandas==1.3.5
requests==2.28.1
numpy==1.20
seldon-core==1.14.1
scikit-learn==1.0.2

Overwriting requirements.txt


## 6.3. Dockerfile Creation

This Dockerfile is comprised of various segments:

* FROM python:3.9-slim: This line selects the official Python 3.9 slim image as the foundational image. Its diminutive size and minimal attack surface amplify both efficiency and security.

* LABEL maintainer="fernandodorado.rueda@ingka.com": A metadata label that designates the image's maintainer, furnishing contact details.

* ENV PYTHONUNBUFFERED=1: By negating Python’s output buffering, log messages are dispatched synchronously, which is conducive to debugging and log assessment.

* WORKDIR /app: Establishes the working directory within the container as /app, serving as a hub for all project components.

* COPY requirements.txt requirements.txt: Incorporates the requirements file into the image. Integrating this prior to the remainder of the code exploits Docker’s caching mechanism, expediting subsequent builds. 

* RUN pip install --no-cache-dir -r requirements.txt: Incorporates the requisite packages as delineated in the requirements file. The --no-cache-dir flag obviates superfluous dependency caching, thereby diminishing the image size.

* COPY DockerModel.py DockerModel.py: Transfers the principal Python file into the image.

* COPY models/ models/: Relocates the model files into the image.

* EXPOSE 5000 9000: Makes ports 5000 (GRPC) and 9000 (REST) accessible, facilitating communication with the encapsulated application.

* ENV MODEL_NAME DockerModel: Designates the environment variable for the model's name.

* ENV SERVICE_TYPE MODEL: Assigns the environment variable pertinent to the service type.

* RUN chown -R 8888 /app: Modifies the directory's ownership to user 8888. Deploying the application under a non-root user account mitigates potential unauthorized write access hazards.

* CMD exec seldon-core-microservice $MODEL_NAME --service-type $SERVICE_TYPE: Initiates the command to activate the service utilizing seldon-core-microservice. It also integrates the model name and service type as arguments. The utilization of exec ascertains that the application accedes to UNIX signals, paving the way for an orderly termination.



In [11]:
%%writefile Dockerfile

# Use an official Python runtime as a parent image.
# Using a slim image for a smaller final size and reduced attack surface.
FROM python:3.9-slim

# Set the maintainer label for metadata.
LABEL maintainer="fernandodorado.rueda@ingka.com"

# Set environment variables for a consistent build behavior.
# Disabling the buffer helps to log messages synchronously.
ENV PYTHONUNBUFFERED=1

# Set a working directory inside the container to store all our project files.
WORKDIR /app

# First, copy the requirements file to leverage Docker's cache for dependencies.
# By doing this first, changes to the code will not invalidate the cached dependencies.
COPY requirements.txt requirements.txt

# Install the required packages listed in the requirements file.
# It's a good practice to include the --no-cache-dir flag to prevent the caching of dependencies
# that aren't necessary for executing the application.
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the code and model files into the image.
COPY DockerModel.py DockerModel.py
COPY models/    models/

# Expose ports that the application will run on.
# Port 5000 for GRPC
# Port 9000 for REST
EXPOSE 5000 9000

# Set environment variables used by the application.
ENV MODEL_NAME DockerModel
ENV SERVICE_TYPE MODEL

# Change the owner of the directory to user 8888 for security purposes.
# It can prevent unauthorized write access by the application itself.
# Make sure to run the application as this non-root user later if applicable.
RUN chown -R 8888 /app

# Use the exec form of CMD so that the application you run will receive UNIX signals.
# This is helpful for graceful shutdown.
# Here we're using seldon-core-microservice to serve the model.
CMD exec seldon-core-microservice $MODEL_NAME --service-type $SERVICE_TYPE


Overwriting Dockerfile


## 6.4 Building Docker Image


Sure, here's the content presented in markdown format:

Building and Pushing Docker Image

1. Installing Docker Desktop. If it hasn't been installed yet, Docker Desktop is the recommended tool for this operation. Docker Desktop provides a GUI (Graphical User Interface) that eases the process of building, initiating, and managing Docker containers. A notable feature of Docker Desktop is its support for Kubernetes, granting a seamless method to form a local cluster.


2. Navigating to the Project Directory. Start by opening a terminal or command prompt. Progress to the directory where the Dockerfile, along with other essential files, is stored.

3. Building the Image. Issue the following command:

```bash
docker build . -t docker-model:1.0.0
```

Here's a breakdown of the command:

* docker build . directs Docker to fabricate the image using the current directory (.).
* -t docker-model:1.0.0 imparts a name (docker-model) and a tag (1.0.0) to the resultant image.

The construction procedure will adhere to the protocols stipulated in the Dockerfile, thus spawning a Docker image that embodies the entire ambiance needed for the model's execution.

In [12]:
!docker build . -t docker-model:5.0.0

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.88kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.9-slim         0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.88kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

# 7.  Deploy ML model using Docker: Unleash it into the world
Once the Docker image has been crafted, the execution phase is relatively straightforward. Let's dissect the procedure step by step:

```bash
docker run --rm --name docker-model -p 9000:9000 docker-model:1.0.0
```

Components of the Command:
docker run: Acts as the foundational command to launch a Docker container.

* --rm: This option ensures that the Docker container undergoes automatic deletion post its cessation. It is especially advantageous for maintaining an uncluttered environment, particularly when initiating containers for ephemeral tasks or testing.

* --name docker-model: Bestows a specific name to the running container.

* -p 9000:9000: This option draws a mapping from port 9000 of the host machine to port 9000 within the Docker container. Presented in the format -p <host_port>:<container_port>, this command is critical given that the Dockerfile specifies the application exposing ports 5000 for GRPC and 9000 for REST. Hence, this step ensures the REST endpoint remains accessible to external applications or users via port 9000 on the host.

docker-model:1.0.0: Pinpoints the name and tag of the Docker image set for execution. Here, docker-model represents the name, while 1.0.0 corresponds to the version tag, as defined during the image construction phase.


In [ ]:
# Execute in a separate terminal if possible
!docker run --rm --name docker-model -p 9000:9000 docker-model:5.0.0

# 8. Test deployed model using Docker
With the Docker image in place, it’s time to see the model in action.

## 8.1. Generate predictions
The path from model to prediction is a delicate process, requiring an understanding of the specific input-output type that Seldon accommodates (e.g., ndarray, JSON data, STRDATA).

In our scenario, the model anticipates an array, and thus, the key in our payload is “ndarray.” Here’s how we orchestrate this:

In [13]:
import requests
import json

URL = "http://localhost:9000/api/v1.0/predictions"

def send_prediction_request(data):
    
    # Create the headers for the request
    headers = {'Content-Type': 'application/json'}

    try:
        # Send the POST request
        response = requests.post(URL, headers=headers, json=data)
        
        # Check if the request was successful
        response.raise_for_status() # Will raise HTTPError if the HTTP request returned an unsuccessful status code
        
        # If successful, return the JSON data
        return response.json()
    except requests.ConnectionError:
        raise Exception("Failed to connect to the server. Is it running?")
    except requests.Timeout:
        raise Exception("Request timed out. Please try again later.")
    except requests.RequestException as err:
        # For any other requests exceptions, re-raise it
        raise Exception(f"An error occurred with your request: {err}")

X_test 

# Define the data payload
data_payload = {
    "data": {
        "ndarray": [
            [
                1.340e+01, 2.052e+01, 8.864e+01, 5.567e+02, 1.106e-01, 1.469e-01,
                1.445e-01, 8.172e-02, 2.116e-01, 7.325e-02, 3.906e-01, 9.306e-01,
                3.093e+00, 3.367e+01, 5.414e-03, 2.265e-02, 3.452e-02, 1.334e-02,
                1.705e-02, 4.005e-03, 1.641e+01, 2.966e+01, 1.133e+02, 8.444e+02,
                1.574e-01, 3.856e-01, 5.106e-01, 2.051e-01, 3.585e-01, 1.109e-01
            ]
        ]
    }
}

# Define the data payload
data_payload = {
    "data": {
        "ndarray": X_test[0:1].tolist()
    }
}

# Get the response and print it
try:
    response = send_prediction_request(data_payload)
    pretty_json_response = json.dumps(response, indent=4)  # Pretty-print JSON
    print(pretty_json_response)
except Exception as err:
    print(err)


{
    "data": {
        "names": [],
        "ndarray": [
            0
        ]
    },
    "meta": {
        "metrics": [
            {
                "key": "accuracy",
                "type": "GAUGE",
                "value": 0
            },
            {
                "key": "precision",
                "type": "GAUGE",
                "value": 0
            },
            {
                "key": "recall",
                "type": "GAUGE",
                "value": 0
            },
            {
                "key": "f1_score",
                "type": "GAUGE",
                "value": 0
            }
        ],
        "tags": {
            "shape_input_data": "(1, 30)"
        }
    }
}


The response from the model will contain several keys:

* "data": Provides the generated output by our model. In our case, it’s the predicted class.
* "meta": Contains metadata and model metrics. It shows the actual values of the classification metrics, including accuracy, precision, recall, and f1_score.
* "tags": Contains intermediate metadata. This could include anything you want to track, such as the shape of the input data.

The structure outlined above ensures that not only can we evaluate the final predictions, but we also gain insights into intermediate results. These insights can be instrumental in understanding predictions and debugging any potential issues.

This stage marks a significant milestone in our journey from training a model to deploying and testing it within a Docker container. We’ve seen how to standardize an ML model and how to set it up for real-world predictions. With this foundation, you’re well-equipped to scale, monitor, and further integrate this model into a full-fledged production environment.




## 8.2. Send feedback in real-time and calculate metrics

The provisioned /feedback endpoint facilitates this learning by allowing truth values to be sent back to the model once they are available. As these truth values are received, the model’s metrics are updated and can be scraped by other tools for real-time analysis and monitoring. In the following code snippet, we iterate over the test dataset and send the truth value to the /feedback endpoint, using a POST request:

In [17]:
import requests
import json

URL = "http://localhost:9000/api/v1.0/feedback"

def send_prediction_feedback(data):
    
    # Create the headers for the request
    headers = {'Content-Type': 'application/json'}

    try:
        # Send the POST request
        response = requests.post(URL, headers=headers, json=data)
        
        # Check if the request was successful
        response.raise_for_status() # Will raise HTTPError if the HTTP request returned an unsuccessful status code
        
        # If successful, return the JSON data
        return response.json()
    except requests.ConnectionError:
        raise Exception("Failed to connect to the server. Is it running?")
    except requests.Timeout:
        raise Exception("Request timed out. Please try again later.")
    except requests.RequestException as err:
        # For any other requests exceptions, re-raise it
        raise Exception(f"An error occurred with your request: {err}")


# payload = {'request': {'data': {'ndarray': [[12.96, 18.29, 84.18, 525.2, 0.07351, 0.07899, 0.04057, 0.01883, 0.1874, 0.05899, 0.2357, 1.299, 2.397, 20.21, 0.003629, 0.03713, 0.03452, 0.01065, 0.02632, 0.003705, 14.13, 24.61, 96.31, 621.9, 0.09329, 0.2318, 0.1604, 0.06608, 0.3207, 0.07247]]}}, 'truth': {'data': {'ndarray': [1]}}}


for i in range(len(X_test)):
    payload = {'request': {'data': {'ndarray': [X_test[i].tolist()]}}, 'truth': {'data': {'ndarray': [int(y_test[i])]}}}

    # Get the response and print it
    try:
        response = send_prediction_feedback(payload)
        pretty_json_response = json.dumps(response, indent=4)  # Pretty-print JSON
        print(pretty_json_response)
    except Exception as err:
        print(err)


{
    "data": {
        "ndarray": []
    },
    "meta": {
        "metrics": [
            {
                "key": "accuracy",
                "type": "GAUGE",
                "value": 0.92621356
            },
            {
                "key": "precision",
                "type": "GAUGE",
                "value": 0.9528302
            },
            {
                "key": "recall",
                "type": "GAUGE",
                "value": 0.9294478
            },
            {
                "key": "f1_score",
                "type": "GAUGE",
                "value": 0.9409938
            }
        ],
        "tags": {
            "shape_input_data": "(1, 30)"
        }
    }
}
{
    "data": {
        "ndarray": []
    },
    "meta": {
        "metrics": [
            {
                "key": "accuracy",
                "type": "GAUGE",
                "value": 0.9263566
            },
            {
                "key": "precision",
                "type": "GAUGE",
         

What makes this approach truly powerful is that the model’s evolution is no longer confined to the training phase. Instead, it’s in a continual state of learning, adjustment, and refinement, based on real-world feedback.

This way, we’re not just deploying a static prediction engine but fostering an evolving intelligent system that can better align itself with the changing landscape of data it interprets. It’s a holistic approach to machine learning deployment that encourages continuous improvement and real-time adaptation.